In [12]:
import os
import random
import shutil

from multiprocessing import cpu_count
from concurrent.futures import ProcessPoolExecutor, as_completed

In [2]:
def scan_files(directory, prefix=None, postfix=None):
    files_list = []
    for root, sub_dirs, files in os.walk(directory):
        for special_file in files:
            if postfix:
                if special_file.endswith(postfix):
                    files_list.append(os.path.join(root, special_file))
            elif prefix:
                if special_file.startswith(prefix):
                    files_list.append(os.path.join(root, special_file))
            else:
                files_list.append(os.path.join(root, special_file))
    return files_list

### partial select

In [ ]:
# 11 yolo classes
yolo_classes = {"ACTINO":9, "CC":8, "VIRUS":10, "FUNGI":6, "TRI":7, "AGC_A":4, 
                "AGC_B":4, "EC":5, "HSIL_B":2, "HSIL_M":2, "HSIL_S":2, "SCC_G":3, 
                "ASCUS":0, "LSIL_F":0, "LSIL_E":1, "SCC_R":3}

not_choose = ["AGC_A", "AGC_B", "HSIL_B", "HSIL_M", "HSIL_S", "FUNGI", "SCC_G", "SCC_R"]
not_choose_index = [yolo_classes[key] for key in not_choose]

In [ ]:
def choose(txt_name):
    with open(txt_name, 'r') as f:
        for line in f.readlines():
            if int(line.strip().split()[0]) in not_choose_index:
                return False
    return True

def move(txt, dst_dir):
#     if txt.endswith("_hls09.txt"):  # these are hl changed, we only need original images
#         continue
    if (not txt.endswith("_hls09.txt")) and choose(txt):
        shutil.move(txt, dst_dir)
        bmp = os.path.splitext(txt)[0] + ".bmp"
        shutil.move(bmp, dst_dir)

def batch_move(txts, dst_dir):
    for txt in txts:
        move(txt, dst_dir)
        
def main(src_dir, dst_dir):
    print("start work")
    txts = scan_files(src_dir, postfix=".txt")
    print("total files", len(txts))
    
    executor = ProcessPoolExecutor(max_workers=4)
    tasks = []

    batch_size = 1000
    for i in range(0, len(txts), batch_size):
        batch = txts[i : i+batch_size]
        tasks.append(executor.submit(batch_move, txts, dst_dir))
    
    job_count = len(tasks)
    for future in as_completed(tasks):
        # result = future.result()  # get the returning result from calling fuction
        job_count -= 1
        print("One Job Done, Remaining Job Count: %s" % (job_count))

In [ ]:
src_dir = "/home/hdd_array0/batch6_1216/train"
dst_dir = "/home/hdd_array0/batch6_1216/train_selected"

main(src_dir, dst_dir)

### select HSIL-SCC_G
 - randomly select half the data from four rotated partitions

In [3]:
classes = set([1])  # HSIL-SCC_G
data_path1 = "/home/ssd_array0/Data/batch6.4_1216/original"
data_path2 = "/home/ssd_array0/Data/batch6.4_1216/rotate"

In [4]:
def read_and_record(txt_names):
    name_map = {}
    
    for i,txt_name in enumerate(txt_names):
        is_in = False
        with open(txt_name, 'r') as f:
            for line in f.readlines():
                tokens = line.strip().split()
                if int(tokens[0]) in classes:
                    is_in = True
                    break

        if is_in:
            basename = os.path.splitext(os.path.basename(txt_name))[0]
            name_map[basename] = [txt_name]
    
        if i % 10000 == 0:
            print(i)
    
    return name_map

In [5]:
files1 = scan_files(data_path1, postfix='.txt')
name_map = read_and_record(files1)

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000
730000
740000
750000
760000
770000
780000
790000
800000
810000
820000
830000
840000
850000
860000
870000
880000
890000
900000
910000
920000
930000
940000
950000
960000
970000
980000
990000
1000000
1010000
1020000


In [6]:
print(len(name_map))

365422


In [7]:
def read_and_record(txt_names, name_map):
    
    for i,txt_name in enumerate(txt_names):
        basename = os.path.splitext(os.path.basename(txt_name))[0]
        basename = basename.split('_90')[0]
        basename = basename.split('_180')[0]
        basename = basename.split('_270')[0]
        if basename in name_map:
            name_map[basename].append(txt_name)
    
        if i % 10000 == 0:
            print(i)
    
    return name_map

In [8]:
files2 = scan_files(data_path2, postfix='.txt')
name_map4 = read_and_record(files2, name_map.copy())

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000
730000
740000
750000
760000
770000
780000
790000
800000
810000
820000
830000
840000
850000
860000
870000
880000
890000
900000
910000
920000
930000
940000
950000
960000
970000
980000
990000
1000000
1010000
1020000
1030000
1040000
1050000
1060000
1070000
1080000
1090000
1100000
1110000
1120000
1130000
1140000
1150000
1160000
1170000
1180000
1190000
1200000
1210000
1220000
1230000
1240000
1250000
1260000
1270000
1280000
1290000
1300000
1310000
1320000
1330000
1340000
1350000
1360000
1370000
1380000
13

In [9]:
print(len(name_map4))

365422


In [10]:
def select_to_delete(name_map, N=2):
    name_map_new = {}
    for name,paths in name_map.items():
        if len(paths) <= N:
            continue
        paths_new = random.sample(paths, N)
        name_map_new[name] = paths_new
    return name_map_new

In [13]:
name_map_to_delete = select_to_delete(name_map4.copy())
print(len(name_map_to_delete))

360809


In [14]:
# flatten path names
paths_to_delete = []
for name,paths in name_map_to_delete.items():
    paths_to_delete += paths
print(len(paths_to_delete))

721618


In [15]:
files1_rest = set(files1) - set(paths_to_delete)
print(len(files1_rest))

842915


In [18]:
files2_rest = set(files2) - set(paths_to_delete)
print(len(files2_rest))

2526256


In [25]:
files1_rest = [os.path.splitext(f)[0]+'.bmp' for f in files1_rest]
files1_lines = '\n'.join(files1_rest) + '\n'
with open('./original.txt', 'w') as f:
    f.write(files1_lines)

In [26]:
files2_rest = [os.path.splitext(f)[0]+'.bmp' for f in files2_rest]
files2_lines = '\n'.join(files2_rest) + '\n'
with open('./rotate.txt', 'w') as f:
    f.write(files2_lines)